In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train=pd.read_csv("/content/train_preprocessed.csv")

In [ ]:
test=pd.read_csv("/content/test_preprocessed.csv")

In [ ]:
train.columns

Index(['Unnamed: 0', 'Subject', 'Body', 'Label', 'SenderEmailAddress', 'url',
       'clean_esc_body', 'Domain_from', 'clean_subject', 'sub_pref_label_0',
       'sub_pref_label_1', 'sub_pref_label_2', 'sub_pref_label_3'],
      dtype='object')

### Normalizing missing Subject and Body in the train and test

In [ ]:
null_from_index=train.loc[train['clean_subject'].isnull()==True].index
for i in null_from_index:
  train.loc[i,'clean_subject']="NOSUB"

In [ ]:
null_from_index=train.loc[train['clean_esc_body'].isnull()==True].index
for i in null_from_index:
  train.loc[i,'clean_esc_body']="NOBOD"

In [ ]:
null_from_index=test.loc[test['clean_subject'].isnull()==True].index
for i in null_from_index:
  test.loc[i,'clean_subject']="NOSUB"

In [ ]:
null_from_index=test.loc[test['clean_esc_body'].isnull()==True].index
for i in null_from_index:
  test.loc[i,'clean_esc_body']="NOBOD"

In [ ]:
null_from_index=train.loc[train['Body'].isnull()==True].index
for i in null_from_index:
  train.loc[i,'Body']="NOBOD"

In [ ]:
null_from_index=test.loc[test['Body'].isnull()==True].index
for i in null_from_index:
  test.loc[i,'Body']="NOBOD"

In [ ]:
train.reset_index(inplace=True)

In [ ]:
train.drop(columns={"index","Unnamed: 0"},inplace=True)

In [ ]:
test.reset_index(inplace=True)

In [ ]:
test.drop(columns={"index","Unnamed: 0"},inplace=True)

### Adding the "$" symbol score.
This tells us if the email has reference to money such as 


1.   Losing Money if action not performed
2.   Rewarding Money if certain action performed



In [ ]:
symbols='$'

def symbol_score(body):
    return sum([char in symbols for char in body])/len(body)

In [ ]:

train['Body'] = train['Body'].astype('string')
train['symbol_score'] = train['Body'].apply(symbol_score)



In [ ]:
test['Body'] = test['Body'].astype('string')
test['symbol_score'] = test['Body'].apply(symbol_score)

### URL COUNT

In [ ]:
train['num_url'] = 0

In [ ]:
train['new_url'] = train['url'].str.strip('[]')

In [ ]:
null_from_index=train.loc[train['new_url']==''].index

for i in null_from_index:
  train.loc[i,'new_url']="NOURL"

In [ ]:
import warnings
warnings.filterwarnings('ignore')
train['num_url'] = 0
con=0
for i in range(len(train)):
    if (train.iloc[i]['new_url']=='NOURL'):
      train['num_url'][i]=0
    else:
    
      train['num_url'][i] = len(train['new_url'][i].split(','))

In [ ]:
#test URL
test['new_url'] = test['url'].str.strip('[]')
null_from_index=test.loc[test['new_url']==''].index

for i in null_from_index:
  test.loc[i,'new_url']="NOURL"


In [ ]:
import warnings
warnings.filterwarnings('ignore')
test['num_url'] = 0
con=0
for i in range(len(test)):
    if (test.iloc[i]['new_url']=='NOURL'):
      test['num_url'][i]=0
    else:
    
      test['num_url'][i] = len(test['new_url'][i].split(','))

### NER

USING SPACY TO PERFORM NER TO UNDERSTAND IF THE EMAIL IS TARGETTING GENERAL USER OR PERSONALIZED TO A USER

In [ ]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

     |████████████████████████████████| 12.0 MB 7.3 MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.0-py3-none-any.whl size=12019121 sha256=36ab1fb171da1d96375198dd9775fbbab9db1c6d887481b4897083bd26515a26
  Stored in directory: /root/.cache/pip/wheels/64/69/41/6f820cf1d7488a0381a2059f66ec9f8f23116f7c67d18f3d8d
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5


In [ ]:
pip install spacy

In [ ]:
import spacy
from spacy import displacy
from collections import Counter

In [ ]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

##PPN IN TRAINING AND TESTIN

In [ ]:
from collections import Counter
ppn_lst=[]
for sub in range(len(train)):
    lst_ner=[]
    #print(sub)
    sub1=train['clean_subject'][sub]
    bod1=train['clean_esc_body'][sub][:51]
    doc1=nlp(sub1)
    doc2=nlp(bod1)
    has_ppn=0
    for ent in doc1.ents:
        lst_ner.append(ent.label_)

        if ent.label_=='PERSON':
          has_ppn=1
    for ent in doc2.ents:
        lst_ner.append(ent.label_)

        if ent.label_=='PERSON':
          has_ppn=1
    ppn_lst.append(has_ppn)

        #print(ent.label_)
    
    #lst_sub.append(Counter(lst_ner))
len(ppn_lst)

5993

In [ ]:
train['ppn_count']=ppn_lst

In [ ]:
#test PPN
from collections import Counter
ppn_lst=[]
for sub in range(len(test)):
    lst_ner=[]
    #print(sub)
    sub1=test['clean_subject'][sub]
    bod1=test['clean_esc_body'][sub][:51]
    doc1=nlp(sub1)
    doc2=nlp(bod1)
    has_ppn=0
    for ent in doc1.ents:
        lst_ner.append(ent.label_)

        if ent.label_=='PERSON':
          has_ppn=1
    for ent in doc2.ents:
        lst_ner.append(ent.label_)

        if ent.label_=='PERSON':
          has_ppn=1
    ppn_lst.append(has_ppn)

        #print(ent.label_)
    
    #lst_sub.append(Counter(lst_ner))
len(ppn_lst)
test['ppn_count']=ppn_lst

##STOPWORDS

In [ ]:

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords    
stop_words = set(stopwords.words('english'))

from nltk.tag import pos_tag
from nltk import word_tokenize
from collections import Counter

In [ ]:
train['text_word_count'] = train['Body'].apply(lambda x: len(str(x).split()))


In [ ]:
stop_word_lst=[]
stop_words = set(stopwords.words('english')) # creating a set makes the searching faster
for i in range(len(train)):
  count=0
  str1=str(train['clean_esc_body'][i])
  str1=str1.split()
  #print("STR1:",str1)
  for word in str1:
    if word in stop_words:
      count=count+1
      #print("this is stop word", word)
  stop_word_lst.append(count)

In [ ]:
train['stop_word_list']=stop_word_lst

In [ ]:
train['body_pct_stop_words'] = train['stop_word_list'] / train['text_word_count']


In [ ]:
#TEST
test['text_word_count'] = test['Body'].apply(lambda x: len(str(x).split()))
stop_word_lst=[]
stop_words = set(stopwords.words('english')) # creating a set makes the searching faster
for i in range(len(test)):
  count=0
  str1=str(test['clean_esc_body'][i])
  str1=str1.split()
  #print("STR1:",str1)
  for word in str1:
    if word in stop_words:
      count=count+1
      #print("this is stop word", word)
  stop_word_lst.append(count)
test['stop_word_list']=stop_word_lst
test['body_pct_stop_words'] = test['stop_word_list'] / test['text_word_count']



# DYNAMIC LINK ANALYSIS TO IDENTIFY MALAICIOUS URLS

In [ ]:
train['Domain_from'].value_counts()

enron               2408
ccac                 157
paypal               147
bankofamerica         65
eftps                 55
                    ... 
alertsp                1
reckittbenckiser       1
technologieinfo        1
case                   1
ostendio               1
Name: Domain_from, Length: 1607, dtype: int64

Common 3rd part vendors list which should be whitelisted

In [ ]:
whitelist=["microsoft.com","outlook.office.com","zoom.us","webex.com","google.com"]

In [ ]:
check_domain=[]
for i in range(len(test)):
  dom=str(train['Domain_from'][i])
  #taking the domain from every row and converting it into string
  l=train['num_url'][i]
  # taking the urls extracted
  label=0
  if l==0:
    label=0
    #if no urls are found give label 0(Flag it as non threat)
  else:
    #case when urls are present
    for j in range(l):
      #loop it through every url
      


      mm=str(train['url'][i].split(',')[j])
      # split the urls by comma
      for wurl in whitelist:
        #loop through whitelisted urls
        p=mm.find(wurl)
        #if substring is not found it returns -1
        if p!=-1:
          #substring found
          label=0
          #the link is safe 
          continue 
      #checking the domain against the urls      
      v=mm.find(dom)            
      if v!=-1:
        # domain found link is safe 
        label=0
      else:
        # link is potentially malicious 
        label=1
        break
        #break the for loop if atleast 1 malicious url
  check_domain.append(label)

  

In [ ]:
train['domain_cont_url']=check_domain

###url domain match checker on TEST

In [ ]:
check_domain=[]
for i in range(len(test)):
  dom=str(test['Domain_from'][i])
  #taking the domain from every row and converting it into string
  l=test['num_url'][i]
  # taking the urls extracted
  label=0
  if l==0:
    label=0
    #if no urls are found give label 0(Flag it as non threat)
  else:
    #case when urls are present
    for j in range(l):
      #loop it through every url
      


      mm=str(test['url'][i].split(',')[j])
      # split the urls by comma
      for wurl in whitelist:
        #loop through whitelisted urls
        p=mm.find(wurl)
        #if substring is not found it returns -1
        if p!=-1:
          #substring found
          label=0
          #the link is safe 
          continue 
      #checking the domain against the urls      
      v=mm.find(dom)            
      if v!=-1:
        # domain found link is safe 
        label=0
      else:
        # link is potentially malicious 
        label=1
        break
        #break the for loop if atleast 1 malicious url
  check_domain.append(label)

  

In [ ]:
test['domain_cont_url']=check_domain

In [ ]:
test['url'].loc[4851]

"['https://ccac', 'https://ccac', 'https://ccac', 'https://ccac', 'http://help.webex.com', 'http://help.webex.com']"

In [ ]:
test[['url','domain_cont_url','Domain_from']].loc[1910]

url                ['https://www.cognitoforms.com']
domain_cont_url                                   1
Domain_from                                    ccac
Name: 1910, dtype: object

In [ ]:
train.to_csv("train_engineered.csv")

In [ ]:
test.to_csv("test_engineered.csv")

# Loading the files again

### EDA

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

In [ ]:
train_eng=pd.read_csv("/content/train_engineered.csv")
test_eng=pd.read_csv("/content/test_engineered.csv")

In [ ]:

train_eng['pronoun_all'] = train_eng['clean_esc_body'].str.lower().str.count('i|me|my|mine|myself|he|him|his|himself|she|her|hers|herself|it|its|itself|they|theirs|their|themself|themselves|we|us|ourselves|you|yours|your|yourself')
train_eng['pronoun_second'] = train_eng['clean_esc_body'].str.lower().str.count('you|yours|your|yourself')
train_eng['percent_fp']=train_eng['pronoun_second']/train_eng['pronoun_all']
train_eng['percent_fp'].fillna(0,inplace=True)

In [ ]:
train_eng['percent_fp']=train_eng['pronoun_second']/train_eng['pronoun_all']

In [ ]:
train_eng.isna().sum()

Unnamed: 0               0
Subject                115
Body                     0
Label                    0
SenderEmailAddress       0
url                      0
clean_esc_body           0
Domain_from              1
clean_subject            0
sub_pref_label_0         0
sub_pref_label_1         0
sub_pref_label_2         0
sub_pref_label_3         0
symbol_score             0
new_url                  0
num_url                  0
ppn_count                0
text_word_count          0
stop_word_list           0
body_pct_stop_words      0
domain_cont_url          0
pronoun_all              0
pronoun_second           0
percent_fp               0
dtype: int64

In [ ]:
train_eng['percent_fp'].fillna(0,inplace=True)

In [ ]:
x1 = train_eng.loc[train_eng['Label']==1]['percent_fp']
x2 = train_eng.loc[train_eng['Label'] ==0]['percent_fp']

group_labels = ['1', '0']

colors = ['rgb(0, 0, 100)', 'rgb(200, 200, 200)']

fig = ff.create_distplot([x1, x2], group_labels,colors=colors)

fig.update_layout(title_text='based on pronouns', template="plotly_white")
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=x1, name='1',
                marker_color = 'rgb(0, 0, 100)'))
fig.add_trace(go.Box(y=x2, name = '2',
                marker_color = 'rgb(200, 200, 200)'))
fig.update_layout(title_text='Box plot of Capital Letter in title', template="plotly_white")
fig.show()

In [ ]:
x1 = train_eng.loc[train_eng['Label']==1]['pronoun_first']
x2 = train_eng.loc[train_eng['Label'] ==0]['pronoun_first']

group_labels = ['1', '0']

colors = ['rgb(0, 0, 100)', 'rgb(200, 200, 200)']

fig = ff.create_distplot([x1, x2], group_labels,colors=colors)

fig.update_layout(title_text='based on pronouns', template="plotly_white")
fig.show()

In [ ]:
x1 = train_eng.loc[train_eng['Label']==1]['pronoun_second']
x2 = train_eng.loc[train_eng['Label'] ==0]['pronoun_second']

group_labels = ['1', '0']

colors = ['rgb(0, 0, 100)', 'rgb(200, 200, 200)']

fig = ff.create_distplot([x1, x2], group_labels,colors=colors)

fig.update_layout(title_text='based on pronouns', template="plotly_white")
fig.show()

In [ ]:
test_samp=pd.read_csv("/content/test_sample_annotated.csv")

In [ ]:
len(test_samp)

201

In [ ]:
test_samp.isna().sum()

Unnamed: 0            0
SenderEmailAddress    0
Subject               0
Body                  0
                      0
clean_esc_body        0
Domain_from           0
clean_subject         0
sub_pref_label_0      0
sub_pref_label_1      0
sub_pref_label_2      0
sub_pref_label_3      0
Label                 0
dtype: int64

In [ ]:
test_samp.dropna(inplace=True)

In [ ]:

test_samp['pronoun_first'] = test_samp['clean_esc_body'].str.lower().str.count('i|me|my|mine|myself|he|him|his|himself|she|her|hers|herself|it|its|itself|they|theirs|their|themself|themselves')
test_samp['pronoun_second'] = test_samp['clean_esc_body'].str.lower().str.count('you|yours|your|yourself')

In [ ]:
x1 = test_samp.loc[test_samp['Label']==1]['pronoun_first']
x2 = test_samp.loc[test_samp['Label'] ==0]['pronoun_first']

group_labels = ['1', '0']

colors = ['rgb(0, 0, 100)', 'rgb(200, 200, 200)']

fig = ff.create_distplot([x1, x2], group_labels,colors=colors)

fig.update_layout(title_text='based on pronouns', template="plotly_white")
fig.show()

In [ ]:
x1 = test_samp.loc[test_samp['Label']==1]['pronoun_second']
x2 = test_samp.loc[test_samp['Label'] ==0]['pronoun_second']

group_labels = ['1', '0']

colors = ['rgb(0, 0, 100)', 'rgb(200, 200, 200)']

fig = ff.create_distplot([x1, x2], group_labels,colors=colors)

fig.update_layout(title_text='based on pronouns', template="plotly_white")
fig.show()

In [ ]:
#body has a lot of persons

In [ ]:
from collections import Counter
ppn_lst_full=[]
for sub in range(len(train_eng)):
    lst_ner=[]
    #print(sub)
    bod1=train_eng['clean_esc_body'][sub]
    doc2=nlp(bod1)
    has_ppn=0
    
    for ent in doc2.ents:
        lst_ner.append(ent.label_)

        if ent.label_=='PERSON':
          has_ppn=has_ppn+1
    ppn_lst_full.append(has_ppn)

        #print(ent.label_)
    
    #lst_sub.append(Counter(lst_ner))
len(ppn_lst_full)

5993

In [ ]:
len(train_eng)

5993

In [ ]:
train_eng['bod_ppn_count']=ppn_lst_full

In [ ]:
train_eng['bod_ppn_count']=train_eng['bod_ppn_count']/train_eng['text_word_count']

In [ ]:
x1 = train_eng.loc[train_eng['Label']==1]['bod_ppn_count']
x2 = train_eng.loc[train_eng['Label'] ==0]['bod_ppn_count']

group_labels = ['1', '0']

colors = ['rgb(0, 0, 100)', 'rgb(200, 200, 200)']

fig = ff.create_distplot([x1, x2], group_labels,colors=colors)
fig.update_xaxes(type="log")

fig.update_layout(title_text='body Person COUNT', template="plotly_white")
fig.show()

In [ ]:
x1 = train_eng.loc[train_eng['Label']==1]['bod_ppn_count']
x2 = train_eng.loc[train_eng['Label'] ==0]['bod_ppn_count']

group_labels = ['1', '0']

colors = ['rgb(0, 0, 100)', 'rgb(200, 200, 200)']

fig = ff.create_distplot([x1, x2], group_labels,colors=colors)

fig.update_layout(title_text='body Person COUNT', template="plotly_white")
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=x1, name='1',
                marker_color = 'rgb(0, 0, 100)'))
fig.add_trace(go.Box(y=x2, name = '2',
                marker_color = 'rgb(200, 200, 200)'))
fig.update_layout(title_text='Box plot of Capital Letter in title', template="plotly_white")
fig.show()